In [ ]:
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Conv3D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import CSVLogger
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import read_csv
import math
import seaborn as sns
from collections import deque
import copy

BATCH_SIZE = 2 
EPOCHS = 1
NUMBER_TEST_SET = 10 #will do train/test split before this, and pass already-made sets in
TEMPORAL_LENGTH = 8 #use first 8 frames (these are 16 frames in each video)

In [ ]:
def file_generator(data_path,data_files,temporal_stride=1,temporal_length=TEMPORAL_LENGTH):
    print(data_files)
    for f in data_files: 
        tmp_df = pd.read_csv(os.path.join(data_path,f), sep=',')
        label_list = list(tmp_df['label'])  
        total_images = len(label_list) 
        if total_images==temporal_length: 
            num_samples = int((total_images-temporal_length)/temporal_stride)+1
            print ('num of frames: {}: {}'.format(f,num_samples))
            img_list = list(tmp_df['FileName'])
        else: # if the number of frames < than temporal length , discard it
            print ('num of frames is less than temporal length; hence discarding this file-{}'.format(f))
            continue

        start_frame = 0
        samples = deque()  
        samp_count=0  
        for img in img_list:
            samples.append(img)
            if len(samples)==temporal_length: 
                samples_c=copy.deepcopy(samples) 
                samp_count+=1
                for t in range(temporal_stride): 
                    samples.popleft()
                yield samples_c,label_list[0]  

In [ ]:
def load_samples(data_cat='train',temporal_stride=1,temporal_length=TEMPORAL_LENGTH):
    data_path = os.path.join('data_files',data_cat)
    data_files = os.listdir(data_path)
    #generator to read the samples
    file_gen = file_generator(data_path,data_files,temporal_stride,temporal_length)
    iterator = True
    data_list = []
    while iterator:
        try:
            x,y = next(file_gen)
            x=list(x)
            data_list.append([x,y])
        except Exception as e:
            print ('Exception: ',e)
            iterator = False
            print ('end of data generator')
    return data_list

In [ ]:
train_data = load_samples(data_cat='train',temporal_stride=1,temporal_length=TEMPORAL_LENGTH)

In [ ]:
test_data = load_samples(data_cat='test',temporal_stride=1,temporal_length=TEMPORAL_LENGTH)

In [ ]:
print ('Total number of train samples:',len(train_data))

In [ ]:
train_data[0]

In [ ]:
print ('Total number of test samples:',len(test_data))

In [ ]:
test_data[0]

In [ ]:
def shuffle_data(samples):
    data = shuffle(samples,random_state=2)
    return data

In [ ]:
def data_generator(data,batch_size=BATCH_SIZE,temporal_padding='same',shuffle=True):               
    num_samples = len(data)
    if shuffle:
        data = shuffle_data(data)
    while True:   
        for offset in range(0, num_samples, batch_size):
            print ('starting index: ', offset) 
            batch_samples = data[offset:offset+batch_size]
            
            X_train = []
            y_train = []
            
            for batch_sample in batch_samples: 
                print(batch_sample)
                # Load image (X)
                x = batch_sample[0] #image
                y = batch_sample[1] #label
                temp_data_list = []
                for img in x:
                    try:
                        img = np.load(img)                        
                        temp_data_list.append(img)
                    except Exception as e:
                        print (e)
                        print ('error reading in frame: ',img)                      

                X_train.append(temp_data_list)
                y_train.append(y)

            X_train = np.array(X_train)   
            y_train = np.array(y_train)
            
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        
            print(X_train.shape)               
            yield X_train, y_train

In [ ]:
print(len(train_data))
print(len(test_data))

train_generator = data_generator(train_data,batch_size=BATCH_SIZE,shuffle=True)

test_generator = data_generator(test_data,batch_size=1,shuffle=False) 
#x,y = next(train_generator)
#xx,yy = next(train_generator)

In [ ]:
def get_model():
    model = Sequential()
    model.add(
    TimeDistributed(
        Conv2D(64, (3,3), activation='relu'), 
        input_shape=(8, 13, 21, 1))
    )
    model.add(
    TimeDistributed(
        Conv2D(64, (3,3), activation='relu')
        )
    )
    model.add(
    TimeDistributed(
        Flatten()
        )
    )        
    model.add(LSTM(1024, activation='relu', return_sequences=False))
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(.01))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='Huber', optimizer=Adam(), metrics=['mean_squared_error'])
    model.summary()
    return model

model = get_model()


In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))    

In [ ]:
checkpoint_path = "cp.ckpt"
earlyStop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=False,
                                                 verbose=1)
csv_logger = CSVLogger('log.csv', append=True, separator=';')

In [ ]:
hist = model.fit(train_generator,
                 steps_per_epoch=(len(train_data)/BATCH_SIZE),
                 epochs=EPOCHS,
                 validation_data=(test_generator), validation_steps=(len(test_data)/BATCH_SIZE),
                 use_multiprocessing=True,workers=6,
                 callbacks=[cp_callback, csv_logger, earlyStop_callback])

In [ ]:
#res = model.predict_generator(test_generator, len(test_data)/BATCH_SIZE)

In [ ]:
test_steps=NUMBER_TEST_SET 
truth = []
pred = []

for i in range(test_steps):
    x, y = next(test_generator)
    pred.append(model.predict(x))
    truth.append(y) 

pred = np.concatenate(pred)
truth = np.concatenate(truth)
print(len(truth))
print(len(pred))

df_predict = pd.DataFrame(pred, columns=['cotBeta'])
df_true = pd.DataFrame(truth, columns=['cotBeta'])
df_predict.to_csv('predictions.csv')
df_true.to_csv('trueLabels.csv')

In [ ]:
sns.distplot(df_true['cotBeta']-df_predict['cotBeta'], kde=False, bins=50)
plt.xlabel('cotBeta residual')
plt.ylabel('frequency')
plt.xlim([-.3,.3])
plt.title('Cot Beta Residual')
plt.savefig('cotBeta-resolution.png')